# Summary of the models that work fine

## Step 0: Get the library and utilities

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.mixture import GMM
%matplotlib inline

In [ ]:
# prediction and saving
def test_save(estimator, test):
    num = test.shape[0]
    test_label = pd.DataFrame({"Id": range(1,num+1), "Solution": estimator.predict(test)}, index=None)
    test_label.to_csv('./test_label-'+str(datetime.date.today())+'.csv', index = None)

## Step 1: Get the data

In [2]:
train = pd.read_csv('train.csv', header=None)
train.columns = range(1, 41)
label = pd.read_csv('trainLabels.csv', header=None)
label.columns = ['label']
test = pd.read_csv('test.csv', header=None)
test.columns = range(1, 41)

## Step 2: GMM + Random Forest
### Step 2.1: GMM

In [ ]:
# Perform GMM on full test data to get clusters. Copy the code from the following reference except the use of aic
# http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html#example-mixture-plot-gmm-selection-py
#x_all = x_train.append(x_test)
x_all = train.append(test)
x_all = x_all.values

lowest_aic = np.infty
aic = []
n_components_range = range(2, 20)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a mixture of Gaussians with EM
        gmm = GMM(n_components=n_components, covariance_type=cv_type)
        gmm.fit(x_all)
        aic.append(gmm.aic(x_all)) # aic is used. Wiki suggests that aic is better than bic
        if aic[-1] < lowest_aic:
            lowest_aic = aic[-1]
            best_gmm = gmm

Step 2.2: Random Forest

In [ ]:
# Fit with randomforest
x_train, x_test, y_train, y_test = cross_validation.train_test_split(train, label, random_state = 0)
x_train_gmm = best_gmm.predict_proba(x_train)
# clf=RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=5, min_samples_split=1,
#  min_samples_leaf=3, max_features='auto',    bootstrap=False, oob_score=False, n_jobs=1, random_state=33,
#  verbose=0)
clf=RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=5)

clf.fit(x_train_gmm, y_train)
clf.score(best_gmm.predict_proba(x_test), y_test)